# Secure Data Access To S3 Bucket via IAM Policy

In [12]:
import boto3
import sagemaker

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

s3 = boto3.Session().client(service_name='s3', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [13]:
role_name = role.split('/')[-1]
print(role_name)

AmazonSageMaker-ExecutionRole-20191006T135881


In [14]:
import time
timestamp = int(time.time())

In [15]:
bucket_secure_name = 'bucket-secure-iam-policy-role-{}'.format(timestamp)
print(bucket_secure_name)

bucket-secure-iam-policy-role-1608485206


In [16]:
!aws s3 mb s3://$bucket_secure_name

make_bucket: bucket-secure-iam-policy-role-1608485206


In [17]:
!aws s3 ls s3://$bucket_secure_name

In [18]:
policy_name='DSOAWS_SecureBucket_Policy_IAM_{}'.format(timestamp)
print(policy_name)

DSOAWS_SecureBucket_Policy_IAM_1608485206


In [19]:
# Create the IAM policy
iam_policy_deny = {
        'Version': '2012-10-17',
        'Statement': [{
            'Sid': '',
            'Effect': 'Deny',
            'Action': [
                's3:ListBucket'
            ],
            'Resource': [
                'arn:aws:s3:::{}'.format(bucket_secure_name)
            ]
        }]
    }

In [20]:
import json 

response = iam.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps(iam_policy_deny)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '4f8df3f2-799e-4ff3-bdc8-251095664bc8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4f8df3f2-799e-4ff3-bdc8-251095664bc8', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sun, 20 Dec 2020 17:26:49 GMT'}, 'RetryAttempts': 0}}


In [21]:
print(bucket_secure_name)

bucket-secure-iam-policy-role-1608485206


In [22]:
!aws s3 ls s3://$bucket_secure_name


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


# Remove the Policy

In [ ]:
response = iam.delete_role_policy(
    RoleName=role_name,
    PolicyName=policy_name
)
print(response)

time.sleep(30)

# Verify Access After Removing the Policy

In [24]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Bucket

In [25]:
!aws s3 rm s3://$bucket_secure_name